In [ ]:
%pip install -q numpy pandas datasets

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [5]:
DATASET_DIR_PATH = "/gdrive/MyDrive/ФТиАД/ВКР/dataset"
FILE_NAME = "lora_train_data.csv"
FEEDBACK_FILE_NAME = "feedback.csv"

RANDOM_STATE = 42

DATASET_SAVEPATH = "/content/lora_train_data"
HF_DATASET_REPO = "il-pugin/hse-porg-task-transformer-lora-dataset"

In [ ]:
import os
import pandas as pd


train_dataset_filepath = os.path.join(DATASET_DIR_PATH, FILE_NAME)
train_dataset_df = pd.read_csv(train_dataset_filepath)
train_dataset_df.head()

In [7]:
import pandas as pd
import json


def format_df(df: pd.DataFrame) -> pd.DataFrame:
  output_df = df[['prompt_id', 'messages']].copy()
  output_df['dataset'] = output_df['prompt_source'] = 'hse-prog-task-transformer'
  output_df["messages"] = output_df["messages"].apply(json.loads)
  return output_df

In [ ]:
formatted_train_dataset_df = format_df(train_dataset_df)


formatted_train_dataset_df.head()

In [ ]:
import os
import pandas as pd


test_dataset_filepath = os.path.join(DATASET_DIR_PATH, FEEDBACK_FILE_NAME)
test_dataset_df = pd.read_csv(test_dataset_filepath)
test_dataset_df.head()

In [11]:
test_dataset_df = test_dataset_df.drop_duplicates(subset='prompt_id')

In [12]:
test_dataset_df = format_df(test_dataset_df)

In [ ]:
test_dataset_df.head()

In [14]:
from datasets import Dataset, DatasetDict


dataset_train = Dataset.from_pandas(formatted_train_dataset_df, preserve_index=False)
dataset_test = Dataset.from_pandas(test_dataset_df, preserve_index=False)

#  Create DatasetDict with standard splits
dataset = DatasetDict({
    "train": dataset_train,
    "test": dataset_test
})



In [ ]:
# Save and upload
dataset.save_to_disk(DATASET_SAVEPATH)

In [17]:
import os
import fnmatch


# These files mess up dataset structure recognition
def delete_state_json_files(dir_path, state_filename: str = 'state.json'):
    for root, dirs, files in os.walk(dir_path):
        for filename in fnmatch.filter(files, state_filename):
            file_path = os.path.join(root, filename)
            os.remove(file_path)
            print(f'Deleted: {file_path}')

In [ ]:
delete_state_json_files(DATASET_SAVEPATH)

In [ ]:
# Upload to Hugging Face Hub
from huggingface_hub import HfApi
from google.colab import userdata


api = HfApi(token=userdata.get("HF_TOKEN"))

# Upload dataset
api.upload_folder(
    folder_path=DATASET_SAVEPATH,
    repo_id=HF_DATASET_REPO,
    repo_type="dataset",
)